# Working with FRB and FRBHost Classes

This notebook demonstrates how to use the core FRB and FRBHost classes to access
FRB data, host galaxy properties, and build tables for population analysis.

In [ ]:
# Standard imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from astropy import units as u
from astropy.coordinates import SkyCoord

## 1. Loading FRBs

The `FRB` class is the primary object for representing Fast Radio Bursts.
The easiest way to load an FRB is by name.

In [ ]:
from frb.frb import FRB

# Load an FRB by name
frb = FRB.by_name('FRB20180924B')
print(frb)

### Accessing FRB Properties

In [ ]:
# Coordinates
print(f"Coordinates: {frb.coord.to_string('hmsdms')}")
print(f"RA: {frb.coord.ra.deg:.4f} deg")
print(f"Dec: {frb.coord.dec.deg:.4f} deg")

In [ ]:
# Dispersion Measure
print(f"DM: {frb.DM}")
print(f"DM ISM (NE2001): {frb.DMISM}")

In [ ]:
# Redshift and Rotation Measure
print(f"Redshift: {frb.z}")
print(f"RM: {frb.RM}")

In [ ]:
# Error ellipse (localization uncertainty)
print("Error Ellipse:")
for key, value in frb.eellipse.items():
    print(f"  {key}: {value}")
    
print(f"\nCombined semi-major axis: {frb.sig_a} arcsec")
print(f"Combined semi-minor axis: {frb.sig_b} arcsec")

In [ ]:
# Other properties
print(f"FRB Name: {frb.frb_name}")
print(f"Repeater: {frb.repeater}")
print(f"References: {frb.refs}")

## 2. Loading Host Galaxies

The `FRBHost` class stores host galaxy properties. The easiest way to access it
is through the FRB object's `grab_host()` method.

In [ ]:
# Get the host galaxy
host = frb.grab_host()
print(host)

### Host Galaxy Properties

In [ ]:
# Basic info
print(f"Host Name: {host.name}")
print(f"Coordinates: {host.coord.to_string('hmsdms')}")
print(f"Redshift: {host.z}")

In [ ]:
# Redshift information
print("Redshift dict:")
for key, value in host.redshift.items():
    print(f"  {key}: {value}")

In [ ]:
# Offsets between FRB and host
print("Offsets:")
for key, value in host.offsets.items():
    if 'ang' in key:
        print(f"  {key}: {value:.3f} arcsec")
    elif 'physical' in key:
        print(f"  {key}: {value:.2f} kpc")

In [ ]:
# Derived quantities (from SED fitting, spectroscopy, etc.)
print("Derived quantities:")
for key, value in host.derived.items():
    if isinstance(value, float):
        if 'Mstar' in key:
            print(f"  {key}: {value:.2e} Msun")
        elif 'SFR' in key:
            print(f"  {key}: {value:.2f} Msun/yr")
        else:
            print(f"  {key}: {value:.3f}")
    else:
        print(f"  {key}: {value}")

In [ ]:
# Photometry
print("\nPhotometry (first 10 entries):")
count = 0
for key, value in host.photom.items():
    if count < 10:
        print(f"  {key}: {value}")
        count += 1

In [ ]:
# Morphology (if available)
if len(host.morphology) > 0:
    print("Morphology:")
    for key, value in host.morphology.items():
        print(f"  {key}: {value}")
else:
    print("No morphology data available")

In [ ]:
# Nebular emission lines (if available)
if len(host.neb_lines) > 0:
    print("Nebular Lines (erg/s/cm^2):")
    for key, value in host.neb_lines.items():
        print(f"  {key}: {value}")
else:
    print("No nebular line data available")

## 3. Building Tables of FRBs

The `build_table_of_frbs()` function creates a pandas DataFrame containing
all FRB properties from the repository.

In [ ]:
from frb.frb import build_table_of_frbs, list_of_frbs

# Build the FRB table
frb_tbl, tbl_units = build_table_of_frbs()

print(f"Total FRBs in repository: {len(frb_tbl)}")
print(f"\nColumns: {frb_tbl.columns.tolist()}")

In [ ]:
# View first few rows
frb_tbl[['FRB', 'RA', 'DEC', 'DM', 'z', 'repeater']].head(10)

In [ ]:
# Check units
print("Units:")
for col in ['DM', 'RA', 'DEC', 'RM', 'ee_a']:
    if col in tbl_units:
        print(f"  {col}: {tbl_units[col]}")

In [ ]:
# FRBs with known redshifts
frbs_with_z = frb_tbl[frb_tbl['z'].notna()]
print(f"FRBs with redshifts: {len(frbs_with_z)}")

In [ ]:
# Repeaters
repeaters = frb_tbl[frb_tbl['repeater'] == True]
print(f"Repeaters: {len(repeaters)}")
print(repeaters['FRB'].values)

In [ ]:
# High DM FRBs
high_dm = frb_tbl[frb_tbl['DM'] > 500]
print(f"FRBs with DM > 500: {len(high_dm)}")

## 4. Building Tables of Host Galaxies

The `build_table_of_hosts()` function creates a pandas DataFrame containing
all host galaxy properties.

In [ ]:
from frb.galaxies import utils as gutils

# Build the host table
host_tbl, host_units = gutils.build_table_of_hosts()

print(f"Total host galaxies: {len(host_tbl)}")
print(f"\nNumber of columns: {len(host_tbl.columns)}")

In [ ]:
# View key columns
key_cols = ['Host', 'FRBname', 'z', 'Mstar', 'SFR_photom']
available_cols = [c for c in key_cols if c in host_tbl.columns]
host_tbl[available_cols].head(10)

In [ ]:
# Hosts with stellar mass measurements
hosts_with_mass = host_tbl[host_tbl['Mstar'].notna()]
print(f"Hosts with Mstar: {len(hosts_with_mass)}")

In [ ]:
# Massive hosts (log Mstar > 10)
if 'Mstar' in host_tbl.columns:
    massive = host_tbl[host_tbl['Mstar'] > 1e10]
    print(f"Hosts with Mstar > 10^10: {len(massive)}")

## 5. Merging FRB and Host Tables

You can merge the FRB and host tables to create a combined dataset.

In [ ]:
# Merge tables on FRB name
merged = pd.merge(frb_tbl, host_tbl, 
                  left_on='FRB', 
                  right_on='FRBname', 
                  how='inner')

print(f"Merged table rows: {len(merged)}")

In [ ]:
# View merged data
cols = ['FRB', 'DM', 'z_x', 'Mstar']
available = [c for c in cols if c in merged.columns]
merged[available].head()

## 6. Simple Visualizations

In [ ]:
# DM distribution
fig, ax = plt.subplots(figsize=(10, 6))
frb_tbl['DM'].hist(bins=20, ax=ax, edgecolor='black', alpha=0.7)
ax.set_xlabel('DM (pc/cm$^3$)', fontsize=12)
ax.set_ylabel('Number of FRBs', fontsize=12)
ax.set_title('Distribution of FRB Dispersion Measures', fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
# DM vs Redshift (Macquart relation)
frbs_z = frb_tbl[frb_tbl['z'].notna()]

if len(frbs_z) > 0:
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.scatter(frbs_z['z'], frbs_z['DM'], s=50, alpha=0.7)
    ax.set_xlabel('Redshift', fontsize=12)
    ax.set_ylabel('DM (pc/cm$^3$)', fontsize=12)
    ax.set_title('DM vs Redshift for Localized FRBs', fontsize=14)
    plt.tight_layout()
    plt.show()

In [ ]:
# Sky distribution
fig = plt.figure(figsize=(12, 6))
ax = fig.add_subplot(111, projection='aitoff')

# Convert to radians for projection
ra_rad = np.deg2rad(frb_tbl['RA'] - 180)  # Center at RA=180
dec_rad = np.deg2rad(frb_tbl['DEC'])

ax.scatter(ra_rad, dec_rad, s=30, alpha=0.7)
ax.grid(True)
ax.set_title('Sky Distribution of FRBs', fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
# Host stellar mass distribution
if 'Mstar' in host_tbl.columns:
    mstar = host_tbl['Mstar'].dropna()
    if len(mstar) > 0:
        fig, ax = plt.subplots(figsize=(10, 6))
        ax.hist(np.log10(mstar), bins=15, edgecolor='black', alpha=0.7)
        ax.set_xlabel('log$_{10}$(M$_*$/M$_\odot$)', fontsize=12)
        ax.set_ylabel('Number of Hosts', fontsize=12)
        ax.set_title('Host Galaxy Stellar Mass Distribution', fontsize=14)
        plt.tight_layout()
        plt.show()

## 7. Listing All FRBs and Hosts

In [ ]:
# List all FRBs
all_frbs = list_of_frbs()
print(f"Total FRBs in repository: {len(all_frbs)}")

# List FRBs with redshifts
frbs_with_z = list_of_frbs(require_z=True)
print(f"FRBs with redshifts: {len(frbs_with_z)}")

In [ ]:
# List all hosts
frbs, hosts = gutils.list_of_hosts(verbose=False)
print(f"Total host galaxies: {len(hosts)}")

# Print summary
for frb_obj, host in zip(frbs[:5], hosts[:5]):
    mstar = host.derived.get('Mstar', 'N/A')
    if isinstance(mstar, float):
        mstar = f"{mstar:.2e}"
    print(f"{frb_obj.frb_name}: z={host.z}, Mstar={mstar}")

## 8. Loading PATH Results

PATH (Probabilistic Association of Transients to Hosts) results are available
for many FRBs.

In [ ]:
# Load PATH table
path_tbl = gutils.load_PATH()
print(f"PATH results available for {len(path_tbl)} FRBs")
print(f"\nColumns: {path_tbl.columns.tolist()}")

In [ ]:
# View PATH results
path_tbl.head()

## Summary

This notebook demonstrated:

1. Loading FRB objects by name using `FRB.by_name()`
2. Accessing FRB properties (coordinates, DM, RM, z, error ellipse)
3. Loading host galaxies using `frb.grab_host()`
4. Accessing host properties (photometry, derived quantities, offsets)
5. Building population tables with `build_table_of_frbs()` and `build_table_of_hosts()`
6. Merging FRB and host data for analysis
7. Simple visualizations of the FRB population

For more details, see:
- FRB class documentation: `frb_class.rst`
- FRBHost class documentation: `frbhost_class.rst`